In [1]:
import pyomo.environ as pyo
import idaes
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes.core.util.scaling as iscale
import ngcc

In [2]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
solver = pyo.SolverFactory("ipopt")

In [3]:
m = pyo.ConcreteModel()
m.fs = ngcc.NgccFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)
#strip_bounds = pyo.TransformationFactory("contrib.strip_var_bounds")
#strip_bounds.apply_to(m, reversible=False)
m.fs.initialize()

2022-03-01 01:13:16 [INFO] idaes.init.fs.fg_translate.properties_in: Starting initialization
2022-03-01 01:13:17 [INFO] idaes.init.fs.fg_translate.properties_in: Property initialization: optimal - Optimal Solution Found.
2022-03-01 01:13:17 [INFO] idaes.init.fs.fg_translate.properties_out: Initialisation Complete, skipped.
2022-03-01 01:13:17 [INFO] idaes.init.fs.fg_translate.properties_out: fs.fg_translate.properties_out State Released.
2022-03-01 01:13:17 [INFO] idaes.init.fs.fg_translate: Initialization Complete optimal - Optimal Solution Found.
2022-03-01 01:13:17 [INFO] idaes.init.fs.st.steam_turbine.inlet_split: Initialization Complete: optimal - Optimal Solution Found
2022-03-01 01:13:18 [INFO] idaes.init.fs.st.steam_turbine.inlet_stage[1]: Initialization Complete: optimal - Optimal Solution Found
2022-03-01 01:13:19 [INFO] idaes.init.fs.st.steam_turbine.inlet_stage[2]: Initialization Complete: optimal - Optimal Solution Found
2022-03-01 01:13:19 [INFO] idaes.init.fs.st.steam_tu

In [4]:
res = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=user-scaling
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale s

In [5]:
jac, nlp = iscale.get_jacobian(m, scaled=True)
print("Extreme Jacobian entries:")
for i in iscale.extreme_jacobian_entries(jac=jac, nlp=nlp, large=100):
    print(f"    {i[0]:.2e}, [{i[1]}, {i[2]}]")
print("Unscaled constraints:")
for c in iscale.unscaled_constraints_generator(m):
    print(f"    {c}")
print("Scaled constraints by factor:")
for c, s in iscale.constraints_with_scale_factor_generator(m):
    print(f"    {c}, {s}")
print("Badly scaled variables:")
for v, sv in iscale.badly_scaled_var_generator(m, large=1e2, small=1e-2, zero=1e-12):
    print(f"    {v} -- {sv} -- {iscale.get_scaling_factor(v)}")
print(f"Jacobian Condition Number: {iscale.jacobian_cond(jac=jac):.2e}")

Extreme Jacobian entries:
    2.49e-05, [fs.gt.cmp1.isentropic[0.0], fs.gt.cmp1.control_volume.properties_in[0.0].log_mole_frac_phase_comp[Vap,CO2]]
    2.49e-05, [fs.gt.cmp1.isentropic[0.0], fs.gt.cmp1.properties_isentropic[0.0].log_mole_frac_phase_comp[Vap,CO2]]
    9.41e-05, [fs.gt.cmp1.isentropic_energy_balance[0.0], fs.gt.cmp1.properties_isentropic[0.0].pressure]
    7.15e-06, [fs.gt.cmp1.control_volume.enthalpy_balances[0.0], fs.gt.cmp1.control_volume.properties_out[0.0].pressure]
    1.33e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air05,CO2], fs.gt.splt1.split_fraction[0.0,air05]]
    1.91e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air05,CO2], fs.gt.splt1.mixed_state[0.0].flow_mol_phase[Vap]]
    1.33e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], fs.gt.splt1.split_fraction[0.0,air07]]
    4.18e-06, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], fs.gt.splt1.mixed_state[0.0].flow_mol_phase[Vap]]
    6.17e-05, [fs.gt.splt1.material_splitting_eqn[0.0,air07,CO2], 

In [6]:
m.fs.net_power.display()

net_power : Size=1
    Key : Value
    0.0 : -650000000.0


In [7]:
m.fs.net_power_mw.display()

net_power_mw : Size=1, Index=fs._time, Units=MW
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :  None :   650 :  None :  True :  True :  Reals


In [8]:
m.fs.lhv_efficiency.display()

lhv_efficiency : Size=1
    Key : Value
    0.0 : 0.5546958107079394


In [9]:
m.fs.aux_compression.display()

aux_compression : Size=1
    Key : Value
    0.0 : 17721366.19459692


In [10]:
m.fs.aux_capture.display()

aux_capture : Size=1
    Key : Value
    0.0 : 10704365.548887009


In [11]:
m.fs.st.hp_steam_temperature.display()

hp_steam_temperature : Size=1, Index=fs._time
    Key : Lower : Value             : Upper : Fixed : Stale : Domain
    0.0 :  None : 856.2959399068224 :  None :  True :  True :  Reals
